# week 5 homework

### setup

In [3]:
import onnx
import torch
import numpy as np
import onnxruntime as ort


from io import BytesIO
from urllib import request
from PIL import Image
from torchvision import transforms


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

!wget {DATA_URL}
!wget {MODEL_URL}

In [18]:
# Q2 - Confirm output name
onnx_model_path = 'hair_classifier_v1.onnx'
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

output_name

'output'

In [14]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)
img = prepare_image(img, target_size=(200, 200))

In [ ]:
# Q3 - Get first pixel
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

x = preprocess(img)
print(f"First pixel, R channel: {x[0, 0, 0].item()}")

First pixel, R channel: -1.0732940435409546


In [ ]:
# Q4 - Model inference
import numpy as np
import onnxruntime as ort

session = ort.InferenceSession('hair_classifier_v1.onnx')
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

x_numpy = x.numpy()
x_numpy = np.expand_dims(x_numpy, axis=0)

result = session.run([output_name], {input_name: x_numpy})
print(f"Model output: {result[0][0][0]}")

Model output: 0.09156641364097595


In [ ]:
# Q5 - Docker image
!docker images

REPOSITORY                       TAG       IMAGE ID       CREATED      SIZE
agrigorev/model-2025-hairstyle   v1        4528ad1525d5   6 days ago   608MB


In [17]:
from test_model import predict

url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
result = predict(url)
print(f"Prediction: {result}")

Prediction: 0.09156641364097595
